In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('tcc_ceds_music.csv')

In [27]:
df

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,82447,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,...,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,82448,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,...,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,82449,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,...,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,82450,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,...,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


In [78]:
new_columns = ['artist_name', 'track_name', 'release_date', 'genre', 'lyrics', 'len', 'topic']

In [79]:
df

,artist_name,track_name,release_date,genre,lyrics,len,topic,age
0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,sadness,1.000000
1,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,world/life,1.000000
2,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,music,1.000000
3,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,romantic,1.000000
4,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,romantic,1.000000
...,...,...,...,...,...,...,...,...
28367,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,obscene,0.014286
28368,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,obscene,0.014286
28369,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,obscene,0.014286
28370,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,obscene,0.014286


In [50]:
df = df[new_columns]

In [31]:
df['release_date'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019], dtype=int64)

In [32]:
df['genre'].unique()

array(['pop', 'country', 'blues', 'jazz', 'reggae', 'rock', 'hip hop'],
      dtype=object)

In [42]:
df['topic'].unique()

array(['sadness', 'world/life', 'music', 'romantic', 'violence',
       'obscene', 'night/time', 'feelings'], dtype=object)

In [41]:
df['len'].max(), df['len'].min()

(199, 1)

In [80]:
def filter_records(df):
    # Display genre choices
    print("Choose a genre:")
    genres = df['genre'].unique()
    for i, genre in enumerate(genres):
        print(f"{i+1}. {genre}")
    
    # Get user genre choice
    genre_choice = input("Enter the number corresponding to your choice: ")
    if not genre_choice.isdigit() or int(genre_choice) < 1 or int(genre_choice) > len(genres):
        print("Invalid choice.")
        return None
    selected_genre = genres[int(genre_choice) - 1]
    print(f"Selected genre: {selected_genre}")

    # Display topic choices
    print("\nChoose a topic:")
    topics = df['topic'].unique()
    for i, topic in enumerate(topics):
        print(f"{i+1}. {topic}")
    
    # Get user topic choice
    topic_choice = input("Enter the number corresponding to your choice: ")
    if not topic_choice.isdigit() or int(topic_choice) < 1 or int(topic_choice) > len(topics):
        print("Invalid choice.")
        return None
    selected_topic = topics[int(topic_choice) - 1]
    print(f"Selected topic: {selected_topic}")

    # Filter the DataFrame based on the user's choices
    filtered_df = df[(df['genre'] == selected_genre) & (df['topic'] == selected_topic)]
    
    print(f"\nNumber of matching records: {len(filtered_df)}")

    
    return filtered_df

# Example usage
filtered_records = filter_records(df)

Choose a genre:
1. pop
2. country
3. blues
4. jazz
5. reggae
6. rock
7. hip hop
Selected genre: blues

Choose a topic:
1. sadness
2. world/life
3. music
4. romantic
5. violence
6. obscene
7. night/time
8. feelings
Selected topic: romantic

Number of matching records: 309


In [83]:
# Assuming filtered_records is your DataFrame containing the filtered records
# and columns is the list of columns to be printed
columns = ['artist_name', 'track_name', 'release_date', 'lyrics']
# Get a random sample of 5 rows
sample_rows = filtered_records.sample(n=5)[columns]

# Print the sample to the user
for index, row in sample_rows.iterrows():
    print(f"Artist: {row['artist_name']}")
    print(f"Track: {row['track_name']} ({row['release_date']})")
    print("Lyrics:")
    print(row['lyrics'])
    print()


Artist: joe cocker
Track: a woman loves a man (1987)
Lyrics:
explain rule game remain mystery mind decide stay surrender away understand love woman woman love woman love best woman love woman love best look eye world hand woman love woman love woman love woman love moon tonight guide stand moon tonight guide stand theres darkness stand stand woman strength conviction truth stronger fiction woman love woman love best woman love woman love best moon tonight guide stand moon tonight guide stand moon tonight guide stand woman love woman love woman love

Artist: the black keys
Track: goodbye babylon (2006)
Lyrics:
come alright desperation suffocation thing goodbye babylon goodbye babylon goodbye babylon know time change people need rearrange write song goodbye babylon goodbye babylon goodbye babylon boys fall leaders appal good go goodbye babylon goodbye babylon goodbye babylon goodbye babylon

Artist: charlie gracie
Track: fabulous (2006)
Lyrics:
know crazy know thrill fabulous lips close 

In [82]:
type(sample_rows)

pandas.core.frame.DataFrame